<a href="https://colab.research.google.com/github/thecodemancer/Apache-Beam/blob/main/learn_beam_windowing_by_doing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

 # **Introduction to Windowing for Batch Processing in Apache Beam**

In this notebook, we will learn the fundamentals of **batch processing** as we walk through a few introductory examples in Beam.
The pipelines in these examples process real-world data for air quality levels in India between 2017 and 2022.

After this tutorial you should have a basic understanding of the following:

*   What is **batch vs. stream** data processing?
*   How can I use Beam to run a **simple batch analysis job**?
*   How can I use Beam's **windowing features** to process only certain intervals of data at a time?

To begin, run the following cell to set up Apache Beam.

In [ ]:
# Install apache-beam.
!pip install --quiet apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 122.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 34.5 MB/s eta 0:00:00


In [ ]:
# Set the logging level to reduce verbose information
import logging

logging.root.setLevel(logging.ERROR)

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Batch vs. Stream Data Processing

What's the difference?

**Batch processing** is when data processing and analysis happens on a set of data that have already been stored over a period of time.
In other words, the input is a finite, bounded data set.
Examples include payroll and billing systems, which have to be processed weekly or monthly.

**Stream processing** happens *as* data flows through a system. This results in analysis and reporting of events
within a short period of time or near real-time on an infinite, unbounded data set.
Examples include fraud detection or intrusion detection, which requires the continuous processing of transaction data.

> This tutorial will focus on **batch processing** examples.
To learn more about stream processing in Beam, check out the [Python Streaming](https://beam.apache.org/documentation/sdks/python-streaming/) page.

<hr style="border: 5px solid #003262;" />

## Load the Data

Let's import the example data we will be using throughout this tutorial. The [dataset](https://www.kaggle.com/datasets/fedesoriano/air-quality-data-in-india)
consists of **hourly air quality data (PM 2.5) in India from November 2017 to June 2022**.

> The World Health Organization (WHO) reports 7 million premature deaths linked to air pollution each year.
In India alone, more than 90% of the country's population live in areas where air quality is below the WHO's standards.

**What does the data look like?**

The data set has 36,192 rows and 6 columns in total recording the following attributes:

1.   `Timestamp`: Timestamp in the format YYYY-MM-DD HH:MM:SS
2.   `Year`: Year of the measure
3.   `Month`: Month of the measure
4.   `Day`: Day of the measure
5.   `Hour`: Hour of the measure
6.   `PM2.5`: Fine particulate matter air pollutant level in air

**For our purposes, we will focus on only the first and last column of the** `air_quality` **DataFrame**:

1.   `Timestamp`: Timestamp in the format YYYY-MM-DD HH:MM:SS
2.   `PM 2.5`: Fine particulate matter air pollutant level in air

Run the following cell to load the data into our file directory.

In [ ]:
# Copy the dataset file into the local file system from Google Cloud Storage.
!mkdir -p data
!gsutil cp gs://batch-processing-example/air-quality-india.csv data/

Copying gs://batch-processing-example/air-quality-india.csv...
/ [1 files][  1.4 MiB/  1.4 MiB]                                                
Operation completed over 1 objects/1.4 MiB.                                      


#### Data Preparation

Before we load the data into a Beam pipeline, let's use Pandas to select two columns.

In [ ]:
# Load the data into a Python Pandas DataFrame.
import pandas as pd

air_quality = pd.read_csv("data/air-quality-india.csv")
air_quality.head()

,Timestamp,Year,Month,Day,Hour,PM2.5
0,2017-11-07 12:00:00,2017,11,7,12,64.51
1,2017-11-07 13:00:00,2017,11,7,13,69.95
2,2017-11-07 14:00:00,2017,11,7,14,92.79
3,2017-11-07 15:00:00,2017,11,7,15,109.66
4,2017-11-07 16:00:00,2017,11,7,16,116.50


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['Hour'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['PM2.5'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_2, *['Hour'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_3, *['PM2.5'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_7, *['PM2.5', 'Timestamp'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_8, *['Timestamp', 'count()', 'Timestamp'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_9, *['Year', 'count()', 'Timestamp'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_10, *['Month', 'count()', 'Timestamp'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_11, *['Day', 'count()', 'Timestamp'], **{})
chart

In [ ]:
import csv

#Select only the two features of the DataFrame we're interested in.
airq = air_quality.loc[:, ["Timestamp", "PM2.5"]].set_index("Timestamp")
saved_new = pd.DataFrame(airq)
saved_new.to_csv("data/air_quality.csv")
airq.head()

,PM2.5
Timestamp,
2017-11-07 12:00:00,64.51
2017-11-07 13:00:00,69.95
2017-11-07 14:00:00,92.79
2017-11-07 15:00:00,109.66
2017-11-07 16:00:00,116.50


<hr style="border: 5px solid #003262;" />

# 1. Average Air Quality Index (AQI)

Before we explore more advanced batch processing with different types of windowing, we will start with a simple batch analysis example.

Our **objective** is to analyze the *entire* dataset to find the **average PM2.5 air quality index** in India across the entire 11/2017-6/2022 period.

> This examples uses the `GlobalWindow`, which is a single window that covers the entire PCollection.
All pipelines use the [`GlobalWindow`](https://beam.apache.org/releases/pydoc/current/apache_beam.transforms.window.html#apache_beam.transforms.window.GlobalWindow) by default.
In many cases, especially for batch pipelines, this is what we want since we want to analyze all the data that we have.

In [ ]:
import apache_beam as beam

def parse_file(element):
  file = csv.reader([element], quotechar='"', delimiter=',',
                    quoting=csv.QUOTE_ALL, skipinitialspace=True)
  for line in file:
    return line

with beam.Pipeline() as pipeline:
 (
      pipeline
      | 'Read input file' >> beam.io.ReadFromText("data/air_quality.csv",
                                                  skip_header_lines=1)
      | 'Parse file' >> beam.Map(parse_file)
      | 'Get PM' >> beam.Map(lambda x: float(x[1])) # only process PM2.5
      | 'Get mean value' >> beam.combiners.Mean.Globally()
      | beam.Map(print))

49.308428658266905


In [ ]:
# To verify, the above mean value matches what Pandas produces
airq["PM2.5"].mean()

49.30842865826703

**Congratulations!** You just created a simple aggregation processing pipeline in batch using `GlobalWindow`.

<hr style="border: 5px solid #003262;" />

# 2. Advanced Processing in Batch with Windowing

Sometimes, we want to [aggregate](https://beam.apache.org/documentation/transforms/python/overview/#aggregation) data, like `GroupByKey` or `Combine`,
only at certain intervals, like hourly or daily, instead of processing the entire `PCollection` of data only once.

In this case, our **objective** is to determine the **fluctuations of air quality *every 30 days*.

**_Windows_** in Beam allow us to process only certain data intervals at a time.
In this notebook, we will go through different ways of windowing our pipeline.

We have already been introduced to the default GlobalWindow (see above) that covers the entire PCollection.
Now we will dive into **fixed time windows, sliding time windows, and session windows**.

> [Another windowing tutorial](https://colab.sandbox.google.com/github/apache/beam/blob/master/examples/notebooks/tour-of-beam/windowing.ipynb) with a toy dataset is recommended to go through.

### First, we need to convert timestamps to Unix time

Apache Beam requires us to provide the timestamp as Unix time. Let us write the simple time conversion function:

In [ ]:
import time

# This function is modifiable and can convert integers to time formats like unix
# Without this function and .strptime, you may run into comparison issues!
def to_unix_time(time_str: str, time_format='%Y-%m-%d %H:%M:%S') -> int:
  """Converts a time string into Unix time."""
  time_tuple = time.strptime(time_str, time_format)
  return int(time.mktime(time_tuple))

In [ ]:
to_unix_time('2021-10-14 14:00:00')

1634220000

### Second, let us define some helper functions to develop our pipeline

In this code, we have a transform (`PrintElementInfo`) to help us analyze an element alongside its window information,
and we have another transform (`PrintWindowInfo`) to help us analyze how many elements landed into each window.
We use a custom [`DoFn`](https://beam.apache.org/documentation/transforms/python/elementwise/pardo)
to access that information.

In [ ]:
#@title Helper functions to develop our pipeline

def human_readable_window(window) -> str:
  """Formats a window object into a human readable string."""
  if isinstance(window, beam.window.GlobalWindow):
    return str(window)
  return f'{window.start.to_utc_datetime()} - {window.end.to_utc_datetime()}'

class PrintElementInfo(beam.DoFn):
  """Prints an element with its Window information for debugging."""
  def process(self, element, timestamp=beam.DoFn.TimestampParam, window=beam.DoFn.WindowParam):
    print(f'[{human_readable_window(window)}] {timestamp.to_utc_datetime()} -- {element}')
    yield element

@beam.ptransform_fn
def PrintWindowInfo(pcollection):
  """Prints the Window information with AQI in that window for debugging."""
  class PrintAQI(beam.DoFn):
    def process(self, mean_elements, window=beam.DoFn.WindowParam):
      print(f'>> Window [{human_readable_window(window)}], AQI: {mean_elements}')
      yield mean_elements

  return (
      pcollection
      | 'Count elements per window' >> beam.combiners.Mean.Globally().without_defaults()
      | 'Print counts info' >> beam.ParDo(PrintAQI())
  )

Note: when you run below code, pay attention to how the human readable windows varies for each window type.

You can also use the built-in [`LogElements`](https://beam.apache.org/releases/pydoc/current/apache_beam.transforms.util.html#apache_beam.transforms.util.LogElements)
PTransform to print the elements with the timestamp and window information.

To illustrate how windowing works, we will use the below toy data:

In [ ]:
# a toy data
air_toy_data = [
    ['2021-10-14 14:00:00', '43.27'],
    ['2021-10-14 15:00:00', '44.17'],
    ['2021-10-14 16:00:00', '48.77'],
    ['2021-10-14 17:00:00', '55.57'],
    ['2021-10-14 18:00:00', '56.95'],
    ['2021-10-21 09:00:00', '36.77'],
    ['2021-10-21 10:00:00', '34.87'],
    ['2021-11-17 01:00:00', '62.64'],
    ['2021-11-17 02:00:00', '65.28'],
    ['2021-11-17 03:00:00', '65.53'],
    ['2021-11-17 04:00:00', '70.18'],
    ['2021-12-11 21:00:00', '69.07'],
    ['2022-01-02 21:00:00', '76.56'],
    ['2022-01-02 22:00:00', '78.77'],
    ['2022-01-02 23:00:00', '73.16'],
    ['2022-01-03 03:00:00', '74.05'],
    ['2022-01-03 19:00:00', '100.28'],
    ['2022-01-03 22:00:00', '80.92'],
    ['2022-01-04 05:00:00', '80.48'],
    ['2022-01-04 07:00:00', '84.0'],
    ['2022-01-04 18:00:00', '95.49'],
    ['2022-01-05 00:00:00', '69.01'],
    ['2022-01-05 07:00:00', '76.85'],]

### Fixed time windows

`FixedWindows` allow us to create fixed-sized windows with evenly spaced intervals.
We only need to specify the _window size_ in seconds.

In Python, we can use [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects)
to help us do the conversion of minutes, hours, or days for us.

We then use the [`WindowInto`](https://beam.apache.org/releases/pydoc/current/apache_beam.transforms.core.html?highlight=windowinto#apache_beam.transforms.core.WindowInto)
transform to apply the kind of window we want.

In [ ]:
import apache_beam as beam
from datetime import timedelta


# We first set the window size to around 1 month.
window_size = timedelta(days=30).total_seconds()  # in seconds


# Let us set up the windowed pipeline and compute AQI every 30 days
with beam.Pipeline() as pipeline:
  (
      pipeline
      | beam.Create(air_toy_data)
      | 'With timestamps' >> beam.MapTuple(
          lambda timestamp, element:
              beam.window.TimestampedValue(float(element), to_unix_time(timestamp))
      )
      | 'Fixed windows' >> beam.WindowInto(beam.window.FixedWindows(window_size))
      | 'Print element info' >> beam.ParDo(PrintElementInfo())
      | 'Print window info' >> PrintWindowInfo()
  )

[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 14:00:00 -- 43.27
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 15:00:00 -- 44.17
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 16:00:00 -- 48.77
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 17:00:00 -- 55.57
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 18:00:00 -- 56.95
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-21 09:00:00 -- 36.77
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-21 10:00:00 -- 34.87
[2021-10-30 00:00:00 - 2021-11-29 00:00:00] 2021-11-17 01:00:00 -- 62.64
[2021-10-30 00:00:00 - 2021-11-29 00:00:00] 2021-11-17 02:00:00 -- 65.28
[2021-10-30 00:00:00 - 2021-11-29 00:00:00] 2021-11-17 03:00:00 -- 65.53
[2021-10-30 00:00:00 - 2021-11-29 00:00:00] 2021-11-17 04:00:00 -- 70.18
[2021-11-29 00:00:00 - 2021-12-29 00:00:00] 2021-12-11 21:00:00 -- 69.07
[2021-12-29 00:00:00 - 2022-01-28 00:00:00] 2022-01-02 21:00:00 -- 76.56
[2021-12-29 00:00:00 - 2022-01-28 00:00:00] 2022-01

### Sliding time windows

[`Sliding windows`](https://beam.apache.org/releases/pydoc/current/apache_beam.transforms.window.html#apache_beam.transforms.window.SlidingWindows)
allow us to compute AQI every 30 days but each window should cover the last 15 days.
We can specify the _window size_ in seconds just like with `FixedWindows` to define the window size.
We also need to specify a _window period_ in seconds, which is how often we want to emit each window.

In [ ]:
import apache_beam as beam
from datetime import timedelta

# Sliding windows of 30 days and emit one every 15 days.
window_size = timedelta(days=30).total_seconds()  # in seconds
window_period = timedelta(days=15).total_seconds()  # in seconds
print(f'window_size:   {window_size} seconds')
print(f'window_period: {window_period} seconds')

with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Air Quality' >> beam.Create(air_toy_data)
      | 'With timestamps' >> beam.MapTuple(
          lambda timestamp, element:
              beam.window.TimestampedValue(float(element), to_unix_time(timestamp))
      )
      | 'Sliding windows' >> beam.WindowInto(
          beam.window.SlidingWindows(window_size, window_period)
      )
      | 'Print element info' >> beam.ParDo(PrintElementInfo())
      | 'Print window info' >> PrintWindowInfo()
  )

window_size:   2592000.0 seconds
window_period: 1296000.0 seconds
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 14:00:00 -- 43.27
[2021-09-15 00:00:00 - 2021-10-15 00:00:00] 2021-10-14 14:00:00 -- 43.27
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 15:00:00 -- 44.17
[2021-09-15 00:00:00 - 2021-10-15 00:00:00] 2021-10-14 15:00:00 -- 44.17
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 16:00:00 -- 48.77
[2021-09-15 00:00:00 - 2021-10-15 00:00:00] 2021-10-14 16:00:00 -- 48.77
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 17:00:00 -- 55.57
[2021-09-15 00:00:00 - 2021-10-15 00:00:00] 2021-10-14 17:00:00 -- 55.57
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-14 18:00:00 -- 56.95
[2021-09-15 00:00:00 - 2021-10-15 00:00:00] 2021-10-14 18:00:00 -- 56.95
[2021-10-15 00:00:00 - 2021-11-14 00:00:00] 2021-10-21 09:00:00 -- 36.77
[2021-09-30 00:00:00 - 2021-10-30 00:00:00] 2021-10-21 09:00:00 -- 36.77
[2021-10-15 00:00:00 - 2021-11-14 00:00:00] 2021-10-21 10:

### Session time windows

Maybe we don't want regular windows, but instead, have the windows reflect periods where activity happened. [`Sessions`](https://beam.apache.org/releases/pydoc/current/apache_beam.transforms.window.html#apache_beam.transforms.window.Sessions)
allow us to create those windows.
We only need to specify a _gap size_ in seconds, which is the maximum number of seconds of inactivity to close a session window.
In this case, if no event happens within 10 days, the current session window closes and
is emitted and a new session window is created whenever the next event happens.

In [ ]:
import apache_beam as beam
from datetime import timedelta

# Sessions divided by 10 days.
gap_size = timedelta(days=10).total_seconds()  # in seconds
print(f'gap_size: {gap_size} seconds')

with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Air Quality' >> beam.Create(air_toy_data)
      | 'With timestamps' >> beam.MapTuple(
          lambda timestamp, element:
              beam.window.TimestampedValue(float(element), to_unix_time(timestamp))
      )
      | 'Session windows' >> beam.WindowInto(beam.window.Sessions(gap_size))
      | 'Print element info' >> beam.ParDo(PrintElementInfo())
      | 'Print window info' >> PrintWindowInfo()
  )

gap_size: 864000.0 seconds
[2021-10-14 14:00:00 - 2021-10-24 14:00:00] 2021-10-14 14:00:00 -- 43.27
[2021-10-14 15:00:00 - 2021-10-24 15:00:00] 2021-10-14 15:00:00 -- 44.17
[2021-10-14 16:00:00 - 2021-10-24 16:00:00] 2021-10-14 16:00:00 -- 48.77
[2021-10-14 17:00:00 - 2021-10-24 17:00:00] 2021-10-14 17:00:00 -- 55.57
[2021-10-14 18:00:00 - 2021-10-24 18:00:00] 2021-10-14 18:00:00 -- 56.95
[2021-10-21 09:00:00 - 2021-10-31 09:00:00] 2021-10-21 09:00:00 -- 36.77
[2021-10-21 10:00:00 - 2021-10-31 10:00:00] 2021-10-21 10:00:00 -- 34.87
[2021-11-17 01:00:00 - 2021-11-27 01:00:00] 2021-11-17 01:00:00 -- 62.64
[2021-11-17 02:00:00 - 2021-11-27 02:00:00] 2021-11-17 02:00:00 -- 65.28
[2021-11-17 03:00:00 - 2021-11-27 03:00:00] 2021-11-17 03:00:00 -- 65.53
[2021-11-17 04:00:00 - 2021-11-27 04:00:00] 2021-11-17 04:00:00 -- 70.18
[2021-12-11 21:00:00 - 2021-12-21 21:00:00] 2021-12-11 21:00:00 -- 69.07
[2022-01-02 21:00:00 - 2022-01-12 21:00:00] 2022-01-02 21:00:00 -- 76.56
[2022-01-02 22:00:00 - 2

Note how the above windows are irregular.

<hr style="border: 5px solid #003262;" />

# 3. Put All Together

Section 2 uses the toy data to go through three different windowing types. Now it is time to analyze the real data (`data/air_quality.csv`).

Can you build one Beam pipeline to compute all AQI values for these windows?


In [ ]:
#@title Edit This Code Cell
...

In [ ]:
# @title Expand to see the answer
import csv
import time
from datetime import timedelta

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib


def to_unix_time(time_str: str, time_format="%Y-%m-%d %H:%M:%S") -> int:
    """Converts a time string into Unix time."""
    time_tuple = time.strptime(time_str, time_format)
    return int(time.mktime(time_tuple))


def human_readable_window(window) -> str:
    """Formats a window object into a human readable string."""
    if isinstance(window, beam.window.GlobalWindow):
        return str(window)
    return f"{window.start.to_utc_datetime()} - {window.end.to_utc_datetime()}"


@beam.ptransform_fn
def OutputWindowInfo(pcollection):
    """Output the Window information with AQI in that window."""

    class GetAQI(beam.DoFn):
        def process(self, mean_elements, window=beam.DoFn.WindowParam):
            yield human_readable_window(window), mean_elements

    return (
        pcollection
        | "Count elements per window"
        >> beam.combiners.Mean.Globally().without_defaults()
        | "Output counts info" >> beam.ParDo(GetAQI())
    )


def parse_file(element):
    file = csv.reader(
        [element],
        quotechar='"',
        delimiter=",",
        quoting=csv.QUOTE_ALL,
        skipinitialspace=True,
    )
    for line in file:
        return line


p = beam.Pipeline()

# get the data
read_text = (
    p
    | "Read input file"
    >> beam.io.ReadFromText("data/air_quality.csv", skip_header_lines=1)
    | "Parse file" >> beam.Map(parse_file)
    | "With timestamps"
    >> beam.MapTuple(
        lambda timestamp, element: beam.window.TimestampedValue(
            float(element), to_unix_time(timestamp)
        )
    )
)

# define the fixed window
window_size = timedelta(days=30).total_seconds()  # in seconds
fixed_window = (
    read_text
    | "Fixed windows" >> beam.WindowInto(beam.window.FixedWindows(window_size))
    | "Output fixed window info" >> OutputWindowInfo()
    | "Write fixed window info"
    >> beam.io.WriteToText("output_fixed", file_name_suffix=".txt")
)

# define the sliding window
window_period = timedelta(days=15).total_seconds()  # in seconds
sliding_window = (
    read_text
    | "Sliding windows"
    >> beam.WindowInto(beam.window.SlidingWindows(window_size, window_period))
    | "Output sliding window info" >> OutputWindowInfo()
    | "Write sliding window info"
    >> beam.io.WriteToText("output_sliding", file_name_suffix=".txt")
)

# define the session window
gap_size = timedelta(days=10).total_seconds()  # in seconds
session_window = (
    read_text
    | "Session windows" >> beam.WindowInto(beam.window.Sessions(gap_size))
    | "Output session window info" >> OutputWindowInfo()
    | "Write session window info"
    >> beam.io.WriteToText("output_session", file_name_suffix=".txt")
)

In [ ]:
# check the entire graph
ib.show_graph(p)

In [ ]:
# run it!
p.run()

In [ ]:
! head -n 5 output_fixed*.txt

('2017-10-21 00:00:00 - 2017-11-20 00:00:00', 98.31031249999997)
('2017-11-20 00:00:00 - 2017-12-20 00:00:00', 90.89567656765671)
('2017-12-20 00:00:00 - 2018-01-19 00:00:00', 100.57643491124253)
('2018-01-19 00:00:00 - 2018-02-18 00:00:00', 77.3498646616541)
('2018-02-18 00:00:00 - 2018-03-20 00:00:00', 67.83549682875258)


In [ ]:
! head -n 5 output_sliding*.txt

('2017-11-05 00:00:00 - 2017-12-05 00:00:00', 98.08713012477715)
('2017-10-21 00:00:00 - 2017-11-20 00:00:00', 98.31031249999997)
('2017-11-20 00:00:00 - 2017-12-20 00:00:00', 90.89567656765678)
('2017-12-05 00:00:00 - 2018-01-04 00:00:00', 92.90070987654323)
('2017-12-20 00:00:00 - 2018-01-19 00:00:00', 100.57643491124259)


In [ ]:
! head -n 5 output_session*.txt

('2017-11-07 12:00:00 - 2018-05-24 06:00:00', 76.37972771641395)
('2018-05-28 10:00:00 - 2022-06-14 15:00:00', 46.04552370042409)


---
If you made it this far, follow [David Regalado](https://beacons.ai/davidregalado) for more code!